In [1]:
#Calling libraries
import urllib
import pandas as pd
import numpy as np

In [2]:
#Retrieving files
urllib.request.urlretrieve("https://github.com/SisonkeBiotik-Africa/AfriBioML/blob/main/XLS/scopus_2020.xls?raw=true", "scopus_2020.xls")
urllib.request.urlretrieve("https://github.com/SisonkeBiotik-Africa/AfriBioML/blob/main/XLS/scopus_2021.xls?raw=true", "scopus_2021.xls")
urllib.request.urlretrieve("https://github.com/SisonkeBiotik-Africa/AfriBioML/blob/main/XLS/scopus_2022.xls?raw=true", "scopus_2022.xls")
urllib.request.urlretrieve("https://github.com/SisonkeBiotik-Africa/AfriBioML/blob/main/XLS/scopus_Until2019.xls?raw=true", "scopus_Until2019.xls")

('scopus_Until2019.xls', <http.client.HTTPMessage at 0x7f06ec03ea90>)

In [3]:
#Creating dataframes
df2020 = pd.read_excel("scopus_2020.xls")
df2021 = pd.read_excel("scopus_2021.xls")
df2022 = pd.read_excel("scopus_2022.xls")
df2019 = pd.read_excel("scopus_Until2019.xls")

In [4]:
#Merging dataframes
df = df2020
df = df.append(df2021, ignore_index=True)
df = df.append(df2022, ignore_index=True)
df = df.append(df2019, ignore_index=True)

In [5]:
df.shape

(3772, 54)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ï»¿Authors                     3772 non-null   object 
 1   Author(s) ID                   3772 non-null   object 
 2   Title                          3772 non-null   object 
 3   Year                           3772 non-null   int64  
 4   Source title                   3772 non-null   object 
 5   Volume                         3045 non-null   object 
 6   Issue                          1552 non-null   object 
 7   Art. No.                       1666 non-null   object 
 8   Page start                     2120 non-null   object 
 9   Page end                       2105 non-null   object 
 10  Page count                     2 non-null      float64
 11  Cited by                       2549 non-null   float64
 12  DOI                            3659 non-null   o

In [7]:
#Eliminating useless columns from the dataframe
considered = np.r_[2:4, 11:13, 15:19, 35, 38, 40, 42]
dfrestricted = df.iloc[:,considered]

In [8]:
dfrestricted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Title                      3772 non-null   object 
 1   Year                       3772 non-null   int64  
 2   Cited by                   2549 non-null   float64
 3   DOI                        3659 non-null   object 
 4   Authors with affiliations  3772 non-null   object 
 5   Abstract                   3772 non-null   object 
 6   Author Keywords            3170 non-null   object 
 7   Index Keywords             3149 non-null   object 
 8   PubMed ID                  948 non-null    float64
 9   Document Type              3772 non-null   object 
 10  Open Access                1945 non-null   object 
 11  EID                        3772 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 353.8+ KB


In [10]:
df_category = pd.read_csv("query.csv")

In [11]:
categories = {}
for i in range(len(df_category)):
  if len(df_category["name"][i]) >= 6: categories[df_category["name"][i]] = df_category["specialtylabel"][i]


In [12]:
with open("categorizedp.csv", "w") as f:
  f.write("Title;EID;DOI;DocumentType;Year;Disease;Category\n")

In [13]:
for i in range(len(df)):
  df["Title"][i] = df["Title"][i].replace(";", ":", df["Title"][i].count(";"))
  for w in categories:
    if (df["Title"][i].lower().find(w.lower()) >= 0): 
      with open("categorizedp.csv", "a") as f1:
        f1.write(df["Title"][i]+";"+df["EID"][i]+";"+str(df["DOI"][i])+";"+df["Document Type"][i]+";"+str(df["Year"][i])+";"+w+";"+categories[w]+"\n")
    if (df["Abstract"][i].lower().find(w.lower()) >= 0): 
      with open("categorizedp.csv", "a") as f1:
        f1.write(df["Title"][i]+";"+df["EID"][i]+";"+str(df["DOI"][i])+";"+df["Document Type"][i]+";"+str(df["Year"][i])+";"+w+";"+categories[w]+"\n")

<ipython-input-13-b6eef7bae3ee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  while (df["Title"][i].find(";") >= 0): df["Title"][i] = df["Title"][i].replace(";", ":")


In [14]:
df_classified = pd.read_csv("categorizedp.csv", sep=";")

In [15]:
df_classified = df_classified.drop_duplicates(keep='first')

In [16]:
len(df_classified["EID"].unique())

3032

In [17]:
df_classified.to_csv("categorizedp.csv", sep=";", index=False)

This SPARQL query has been used to extract a lexicon aligned to medical specialties from Wikidata: https://w.wiki/6BdE

```
SELECT ?name ?specialtylabel WHERE {
  {?disease rdfs:label ?name} UNION {?disease skos:altLabel ?name}
  FILTER(LANG(?name)="en")
  ?disease wdt:P1995 ?specialty.
  ?specialty rdfs:label ?specialtylabel.
  FILTER(LANG(?specialtylabel)="en")
  }
```

